In [ ]:
%pip install -r "../requirements.txt"

In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

In [ ]:
!sudo nvidia-smi --gpu-reset -i 0
!nvidia-smi

In [ ]:
import torch

torch.cuda.reset_max_memory_allocated()

In [ ]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

In [ ]:
!pip install transformers==4.40.1 peft==0.4.0
!pip install sentencepiece
!pip install accelerate
!pip install torch
!pip install peft
!pip install datasets
!pip install bitsandbytes


In [ ]:
%pip install dotenv
%pip install accelerate

# FinGPT Sentiment Analysis

In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import torch
import gc

def print_gpu_memory():
    allocated = torch.cuda.memory_allocated() / (1024**2)
    cached = torch.cuda.memory_reserved() / (1024**2)
    print(f"Allocated: {allocated:.2f} MB")
    print(f"Cached: {cached:.2f} MB")

# Before clearing the cache
print("Before clearing cache:")
print_gpu_memory()

# Clearing cache
gc.collect()
torch.cuda.empty_cache()

# After clearing the cache
print("\nAfter clearing cache:")
print_gpu_memory()

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [ ]:
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, LlamaTokenizerFast
from peft import PeftModel  # 0.5.0
import torch
import os
from dotenv import load_dotenv
from transformers import BitsAndBytesConfig

load_dotenv(
    dotenv_path="../.env"
)
huggingface_token = os.getenv("FINGPT_ACCESS_TOKEN")

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True, 
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=True,
)

# Load Models
base_model = "meta-llama/Meta-Llama-3-8B" 
peft_model = "FinGPT/fingpt-mt_llama3-8b_lora"
tokenizer = LlamaTokenizerFast.from_pretrained(
    base_model, 
    trust_remote_code=True, 
    token=huggingface_token,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
)
tokenizer.pad_token = tokenizer.eos_token
model = LlamaForCausalLM.from_pretrained(base_model, trust_remote_code=True, device_map = "cuda:0")
model = PeftModel.from_pretrained(model, peft_model)
model = model.eval()


In [ ]:
# Make prompts
prompt = [
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .
Answer: ''',
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
Answer: '''
]

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
tokens = tokenizer(prompt, return_tensors='pt', padding=True, max_length=512).to(device)
res = model.generate(**tokens, max_length=512)
res_sentences = [tokenizer.decode(i) for i in res]
out_text = [o.split("Answer: ")[1] for o in res_sentences]


# Show results
for sentiment in out_text:
    print(sentiment)